# Denoising Autoencoder 

    1. Tried but the model is not as useful as expected. 
    2. Not used in any analysis (only in the section Fixing Mistakes)

In [ ]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

import copy

import numpy as np
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

from sklearn.model_selection import KFold, train_test_split
import matplotlib.pyplot as plt


### The Forward pass of NN would output x reconstructed as well as the label. 

    1. During pretraining the input is corrupted data (call noisy_encode)
    2. During finetuning the input is not corrupted. 

In [ ]:
class Simple_NN(nn.Module):
    def __init__(self, input_dim, hidd_dim, dropout_rate=0.8, denoise_dropout=0.8):
        super(Simple_NN, self).__init__()
        
        self.linear1 = nn.Sequential(
                            nn.Linear(input_dim, hidd_dim), 
                            nn.Dropout(p=dropout_rate), 
                            nn.ReLU())
        
        self.linear2 = nn.Sequential(
                            nn.Linear(hidd_dim, hidd_dim), 
                            nn.Dropout(p=dropout_rate) , 
                            nn.ReLU())
        
        self.linear3 = nn.Sequential(
                            nn.Linear(hidd_dim, hidd_dim), 
                            nn.Dropout(p=dropout_rate) , 
                            nn.ReLU())

        self.linear_final = nn.Linear(hidd_dim, 2)
        
        self.linear_decode = nn.Sequential( nn.Linear(hidd_dim, input_dim), 
                                           nn.Sigmoid())
        
        self.drop =  nn.Dropout(p=denoise_dropout) 

    def forward(self, x): 
        x = self.linear1(x)
        x = self.linear2(x)
        x = self.linear3(x)
        
        x_recon = self.linear_decode(x)
        x_label = self.linear_final(x)
        
        return x_recon, x_label
    
    def noisy_encode(self, x): 
        x = self.drop(x)
        return self.forward(x)
        


In [ ]:
from data_processing import *

train_path = '../data/coupons/train.csv'
test_path = '../data/coupons/test.csv'
sample_path = '../data/coupons/sample_submission1.csv'

data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion.to(device)

criterion_recon = nn.MSELoss()
criterion_recon.to(device)


In [ ]:
drop_first = False
prune=False

X_df, y_df = clean_all(data, drop_first, prune)

X = np.array(X_df)
y = np.array(y_df)


X_train_np, X_val_np, y_train_np, y_val_np = train_test_split(X, y, test_size=0.2, random_state=1)



In [ ]:
X_full = torch.tensor(X).type(torch.FloatTensor)
y_full = torch.tensor(y)


X_train = torch.tensor(X_train_np).type(torch.FloatTensor)
X_val = torch.tensor(X_val_np).type(torch.FloatTensor)

y_train = torch.tensor(y_train_np)
y_val = torch.tensor(y_val_np)

full_dataset = TensorDataset(X_full, y_full) # create your datset
full_dataloader =  DataLoader(dataset=full_dataset, batch_size=64, shuffle=True)


train_dataset = TensorDataset(X_train, y_train) # create your datset
train_dataloader =  DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

eval_dataset = TensorDataset(X_val, y_val) # create your datset
eval_dataloader =  DataLoader(dataset=eval_dataset, batch_size=512, shuffle=True)


In [ ]:
def eval_model(eval_dataloader, model):
    pred = []
    true = []
    eval_loss = 0 
    
    model.eval()
    for i, (x, y) in enumerate(eval_dataloader): 
        x = x.to(device)
        y = y.to(device)
        
        with torch.no_grad(): 
            x_recon, x_label = model(x) 
            loss = criterion(x_label, y)
            eval_loss += loss 
            pred_labels = torch.argmax(x_label, axis=1)
            
            pred.extend(pred_labels)
            true.extend(y)
        
    pred = torch.tensor(pred).cpu().numpy()
    true = torch.tensor(true).cpu().numpy()
    
    eval_loss = eval_loss/(i+1)
    
    acc = np.mean(pred == true)
    model.train()
    return acc, eval_loss
    


In [ ]:
def test_model(test_dataloader, model):
    pred = []
    
    model.eval()
    for i, (x, y) in enumerate(test_dataloader): 
        x = x.to(device)
        
        with torch.no_grad(): 
            x_recon, x_label = model(x) 
            pred_labels = torch.argmax(x_label, axis=1)
            
            pred.extend(pred_labels)
        
    pred = torch.tensor(pred).cpu().numpy()
    model.train()
    return pred
    


## Pretraining 

In [ ]:
model = Simple_NN(input_dim=110, hidd_dim=1000, dropout_rate=0.5, denoise_dropout=0.8)
model.to(device)

optimizer = torch.optim.Adam(model.parameters())


for epoch in range(50): 
    model.train()
    total_loss = 0 
    for i, (x, y) in enumerate(train_dataloader): 
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        x_recon, x_label = model.noisy_encode(x) 
        
        loss_label = criterion(x_label, y)
        loss_recon = criterion_recon(x_recon, x)
        
        loss = loss_recon

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        

## Finetune

In [ ]:
train_losses = []
eval_losses = []
eval_acc = []


for epoch in range(100): 
    model.train()
    total_loss = 0 
    for i, (x, y) in enumerate(train_dataloader): 
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        x_recon, x_label = model(x) 
        
        loss_label = criterion(x_label, y)
        loss = loss_label

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
    total_loss = total_loss/(i+1)
    acc, eval_loss = eval_model(eval_dataloader, model)
    
    train_losses.append(total_loss)
    eval_losses.append(eval_loss)
    eval_acc.append(acc)
    
print('Max Eval Acc: ', np.max(eval_acc))
print('Final Eval accuracy:', eval_acc[-1] )


In [ ]:
from scipy import stats


eval_analysis_dataloader =  DataLoader(dataset=eval_dataset, batch_size=1, shuffle=False)


pred = []
true = []
eval_loss = 0 

model.eval()

all_loss_label = []
all_loss_recon = []

all_pred = []
all_label = []


for i, (x, y) in enumerate(eval_analysis_dataloader): 
    x = x.to(device)
    y = y.to(device)

    with torch.no_grad(): 
        
        x_recon, x_label = model(x) 
        loss_label = criterion(x_label, y)
        loss_recon = criterion_recon(x_recon, x)
        
        pred_labels = torch.argmax(x_label, axis=1)
        all_pred.extend(pred_labels)
        all_label.extend(y)
                        
        all_loss_label.append(loss_label.item() )
        all_loss_recon.append(loss_recon.item() )
            
            
all_loss_label = torch.tensor(all_loss_label).cpu().numpy()
all_loss_recon = torch.tensor(all_loss_recon).cpu().numpy()

all_pred = torch.tensor(all_pred).cpu().numpy()
all_label = torch.tensor(all_label).cpu().numpy()



stats.pearsonr(all_loss_label, all_loss_recon)